- From Chat GPT:
    - Need to use Tika's image recognition parsers, which are powered by Deeplearning4j (DL4J) and can detect a thousand different objects in images using pre-trained models like InceptionNet-V3 and VGG-16.
    - To get started, you will need to download and install Apache Tika and its dependencies (tika-parsers and tika-dl), and configure Tika to use either the InceptionNet-V3 or VGG-16 model in a tika-config.xml file. You can then use Tika's Java API to parse images and retrieve the detected objects from the metadata.

- From Mattman
    - Don't build the xml file, just use the docker
    - Run this: docker run -it -p 8764:8764 uscdatascience/inception-rest-tika
    - This is how to test it: http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://upload.wikimedia.org/wikipedia/commons/f/f6/Working_Dogs%2C_Handlers_Share_Special_Bond_DVIDS124942.jpg
    - more documentation here: https://cwiki.apache.org/confluence/display/TIKA/TikaAndVision#TikaAndVision-a.Usingdocker(Recommended)
    - to find image caption, use this docker: docker run -it -p 8764:8764 uscdatascience/im2txt-rest-tika
    - see this: see this: https://github.com/USCDataScience/tika-dockers
    - then see this: http://localhost:8764/inception/v3/caption/image?url=https://upload.wikimedia.org/w[…]s_Thames_Tigers_2007.jpg/1200px-Marcus_Thames_Tigers_2007.jpg
    - additional documentation: https://cwiki.apache.org/confluence/display/TIKA/ImageCaption#ImageCaption-a.Usingdocker(Recommended)




In [17]:
import tika
import tika.parser
from tika import parser


In [24]:
#This isn't working, from chat GPT. Need to find a way to use the docker

# Parse the image using Tika with Tensorflow recognition
tika.initVM()
parsed = parser.from_file('95k_Images/Pixstory-image-164268756916.jpeg', config_path='tika-config.xml')
results = parsed['metadata']#['Content-Type']


print(results)




{'Comments': 'CREATOR: gd-jpeg v1.0 (using IJG JPEG v80), quality = 90\n', 'Component 1': 'Y component: Quantization table 0, Sampling factors 1 horiz/1 vert', 'Component 2': 'Cb component: Quantization table 1, Sampling factors 1 horiz/1 vert', 'Component 3': 'Cr component: Quantization table 1, Sampling factors 1 horiz/1 vert', 'Compression Type': 'Baseline', 'Content-Type': 'image/jpeg', 'Data Precision': '8 bits', 'File Modified Date': 'Mon Mar 27 02:28:49 +00:00 2023', 'File Name': 'apache-tika-4708901539691810768.tmp', 'File Size': '48955 bytes', 'Image Height': '299 pixels', 'Image Width': '400 pixels', 'JPEG Comment': 'CREATOR: gd-jpeg v1.0 (using IJG JPEG v80), quality = 90', 'Number of Components': '3', 'Number of Tables': '4 Huffman tables', 'Resolution Units': 'inch', 'Thumbnail Height Pixels': '0', 'Thumbnail Width Pixels': '0', 'X Resolution': '96 dots', 'X-Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.ocr.TesseractOCRParser', 'org.apache.ti

In [ ]:
http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://upload.wikimedia.org/wikipedia/commons/f/f6/Working_Dogs%2C_Handlers_Share_Special_Bond_DVIDS124942.jpg